In [1]:
import time
import numpy as np
from datetime import datetime
from calendar import timegm
from pymongo import MongoClient, ASCENDING, DESCENDING, ReturnDocument
import pyvisa
import serial
from pprint import pprint

## Connect to Database

In [2]:
dbaddr = "mongodb://prl-redmine.syr.edu"
def dbConnect():
    client = MongoClient(dbaddr)
    collection = client.data.adr2datas
    infocollection = client.control.adr2controls
    jobcollection = client.jobs.adr2jobs
    return collection, infocollection, jobcollection
collection,infocollection, jobcollection = dbConnect()

## Connect to Instruments

In [4]:
def connectInstruments():
    rm = pyvisa.ResourceManager()
    rm.list_resources()
#     inst1 = rm.open_resource('GPIB0::1::INSTR', read_termination = '\x00')
#     inst2 = rm.open_resource('GPIB0::2::INSTR', read_termination = '\x00')
#     inst3 = rm.open_resource('GPIB0::6::INSTR', read_termination = '\x00')
    
    inst1 = rm.open_resource('GPIB0::1::INSTR')
    inst2 = rm.open_resource('GPIB0::2::INSTR')
    inst3 = rm.open_resource('GPIB0::6::INSTR')
    return [inst1,inst2,inst3]

## Create Information and Data Entries (DO NOT RUN THESE)

In [6]:
def get_info():
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "fridgeStatus": "Warm",
        "currentJob": "Magup",
        "switchState": "Closed",
        "currentLimit": 9.2,
        "maxVoltage": 500,
        "voltageStep": 1,
        "command": "None",
        "jobStart": timegm(datetime.now().timetuple())
    }
    return post
get_info()

{'command': 'None',
 'currentJob': 'Magup',
 'currentLimit': 9.2,
 'fridgeStatus': 'Warm',
 'jobStart': 1459254916,
 'maxVoltage': 500,
 'switchState': 'Closed',
 'timeStamp': 1459254916,
 'voltageStep': 1}

In [9]:
infocollection.insert(get_info())

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':


ObjectId('56faaef16545eb19194e34d2')

In [12]:
command = 'Magup'
infocollection.find_one_and_update({},{'$set': {'command': command}})

{u'_id': ObjectId('56f950d56545eb19194e34d1'),
 u'command': u'Magup',
 u'currentJob': u'Magup',
 u'currentLimit': 9.2,
 u'fridgeStatus': u'Magup',
 u'jobStart': 1459250107,
 u'maxVoltage': 400,
 u'switchState': u'Closed',
 u'timeStamp': 1459165333,
 u'voltageStep': 1}

In [13]:
maxVoltage = 550
infocollection.find_one_and_update({},{'$set': {'maxVoltage': maxVoltage}})

{u'_id': ObjectId('56f950d56545eb19194e34d1'),
 u'command': u'Magup',
 u'currentJob': u'Magup',
 u'currentLimit': 9.2,
 u'fridgeStatus': u'Magup',
 u'jobStart': 1459250107,
 u'maxVoltage': 400,
 u'switchState': u'Closed',
 u'timeStamp': 1459165333,
 u'voltageStep': 1}

## Get Data from Instruments

In [6]:
inst = connectInstruments()

In [12]:
inst[0].query('*IDN?')

u'LSCI,MODEL340,341911,042304\n'

In [17]:
def get_data(inst):
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "baseTemp": float(inst[1].ask('KRDG? B')),
        "oneKTemp": float(inst[1].ask('KRDG? A')),
        "threeKTemp": float(inst[0].ask('KRDG? A')),
        "sixtyKTemp": float(inst[0].ask('KRDG? B')),
        "magnetVoltage": float(inst[0].ask('SRDG? D')),
        "psVoltage": float(inst[2].ask('MEAS:VOLT?')),
        "psVoltCommand": float(inst[2].ask('VOLT?')),
        "psCurrent": float(inst[2].ask('MEAS:CURR?')),
        "currentJob": "None",
        "percentComplete": round(float(inst[2].ask('MEAS:CURR?'))/9 * 100),
        "switchState": "Closed"
    }
    return post
get_data(inst)

{'baseTemp': 2.827,
 'currentJob': 'None',
 'magnetVoltage': 0.00091,
 'oneKTemp': 2.775,
 'percentComplete': 0.0,
 'psCurrent': 0.0285675,
 'psVoltCommand': 0.004,
 'psVoltage': -0.00503014,
 'sixtyKTemp': 48.253,
 'switchState': 'Closed',
 'threeKTemp': 2.967,
 'timeStamp': 1450177941}

In [18]:
get_data(inst)

{'baseTemp': 2.841,
 'currentJob': 'None',
 'magnetVoltage': 0.00091,
 'oneKTemp': 2.775,
 'percentComplete': 0.0,
 'psCurrent': 0.0285675,
 'psVoltCommand': 0.004,
 'psVoltage': -0.00503014,
 'sixtyKTemp': 48.253,
 'switchState': 'Closed',
 'threeKTemp': 2.9671,
 'timeStamp': 1450177944}

## PID

In [5]:
#########################################################################
Pc=0.01            #PID paramters
Ic=0.0
Dc=2.5
dt=5   #wait time
########################################################################

#########################################################################
##################### SAFETY LIMITS  #################################### 
maxstep=5    # max volatge step    #limits
maxvol=150e-3     # max capaitor volatge 


#########################################################################





###################   SET VALUE  #######################################
sv=.5  #set value
apps=1e-3                  #accuracy for set value
########################################################################


def prop(err):
        
    return Pc*err[-1]

def integ(err,tim,dt,Ic):
    
    Ierr=err[-1]*dt  
 
    return Ic*Ierr

def diffren(err):
    Derr=0
    if err.shape[0]>1:
        Derr=(err[-1]-err[-2])/dt
        
    return Dc*Derr

## Setup Log File

In [20]:
logFileName = 'ADR2Log.txt'
inst = connectInstruments()
ps = inst[2]
currentJob = None
statusString = ""
err = []

## Main Loop

In [23]:
while True:
    # Open Log File
    logFile = open(logFileName, 'a')

    # Read Data
    try:
        data = get_data(inst)

    # Log errors if they occur and reconnect
    except UnicodeDecodeError:
        string = "Read error at " + datetime.now().strftime('%H:%M:%S') + '\n'
        print(string)
        logFile.write(string)
        inst = connectInstruments()

    # Insert Data into Database
    else:
        if (data['sixtyKTemp'] <= 300):
            try:
                collection.insert(data)
            except:
                print("Database error at " + datetime.now().strftime('%H:%M:%S'))
    
    # Get information from Database

    info = infocollection.find().next()

    if info['command'] == 'Magup':
        #Ensure fridge is actually cold enough to mag up!
        if data['threeKTemp'] > 4.5:
            pass
        elif info['currentJob'] != 'Magup':
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magup', 'jobStart': data['timeStamp']}})
            logString = "Start Magup to " + str(info['maxVoltage']) + "mV at " + datetime.now().strftime('%a, %c')
            print(logString)
            logFile.writelines([logString, '\n'])

    elif info['command'] == 'Magdown':
        if info['currentJob'] != 'Magdown':
            if info['currentJob'] == 'Soak':
                stopSoakTime = datetime.now()
                t = (stopSoakTime - startSoakTime)
                logString = "Soaked for {} days, {}h: {}m: {}s".format(t.days,t.seconds//3600,(t.seconds//60)%60, t.seconds%60)
                print(logString)
                logFile.writelines([logString, '\n'])
            
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magdown', 'jobStart': data['timeStamp']}})
            logString = "Start Magdown at " + datetime.now().strftime('%a, %c')
            print(logString)
            logFile.writelines([logString, '\n'])

    elif info['command'] == 'Soak':
        if info['currentJob'] != 'Soak':
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Soak', 'jobStart': data['timeStamp']}})
            logString = "Start Soak at " + datetime.now().strftime('%a, %c')
            startSoakTime = datetime.now()
            print(logString)
            logFile.writelines([logString, '\n'])

    else:
        info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'None'}})
        if data['baseTemp'] < 3 and data['baseTemp'] != 0.0:
            statusString = "Cold (Magged Down)"
        elif data['threeKTemp'] < 4.5:
            statusString = "Cold (3K)"
        elif (data['sixtyKTemp'] < data['threeKTemp']) and (data['sixtyKTemp'] > 50):
            statusString = "Cooling Down"
        elif (data['threeKTemp'] > 285):
            statusString = "Warm"
        elif (data['sixtyKTemp'] > data['threeKTemp']) and (data['sixtyKTemp'] > 60):
            statusString = "Warming Up"
        else:
            statusString = "Uncertain State"
        
    # Control Magnet
    job = info['currentJob']

    if job == 'Magup':
        # Increase Magnet Voltage
        currentV = data['psVoltCommand']*1000
        if currentV < info['maxVoltage']:
            nextV = currentV + info['voltageStep']
            ps.write('VOLT:LEV:IMM ' + str(nextV) + 'mV')
            print(str(nextV) + "mV\t" + "Current: " + str(data['psCurrent']) + "\tVoltage: " + str(data['psVoltage']))
        else:
#             print('Max Voltage Reached')
            if data['psCurrent'] > 9.0:
                print('9A reached')
                info = infocollection.find_one_and_update({},{'$set': {'command': 'Soak'}})
        statusString = "Magup"
        
        
#         # Increase Magnet Voltage
#         currentV = float(ps.ask('VOLT?'))*1000
#         cerr = 9.1 - data['psCurrent']
#         err = np.append(err, cerr)
        
#         if np.fabs(cerr) < .01:
#             print('Magup Complete')
#             info = infocollection.find_one_and_update({},{'$set': {'command': 'Soak'}})
            
#         else:
#             nextV = prop(err) + diffren(err)
#             if np.fabs(nextV) > maxstep:
#                 nextV = nextV/np.fabs(nextV) * maxstep
                
#             if data['magnetVoltage'] > 0.7*maxvol:
#                 nextV = nextV*0.5
#             if data['magnetVoltage'] > 0.8*maxvol:
#                 nextV = 0
                
#             nextV = currentV + nextV
# #             ps.write('VOLT:LEV:IMM ' + str(nextV) + 'mV')
#             print(str(nextV) + "mV\t" + "Current: " + ps.ask('MEAS:CURR?') + "\tVoltage: " + ps.ask('MEAS:VOLT?'))
         
        
    elif job == 'Magdown':
        # Decrease Magnet Voltage
        currentV = data['psVoltCommand']*1000
        if currentV > 0:
            nextV = currentV - info['voltageStep']
            ps.write('VOLT:LEV:IMM ' + str(nextV) + 'mV')
            print(str(nextV) + "mV\t" + "Current: " + str(data['psCurrent']) + "\tVoltage: " + str(data['psVoltage']))
        else:
#             print('Zero Voltage Reached')
            info = infocollection.find_one_and_update({},{'$set': {'command': 'None'}})
        statusString = "Magdown"
    elif job == 'Soak':
        # Soak
        statusString = "Soak"
    else:
        pass
    
    info = infocollection.find_one_and_update({},{'$set': {'fridgeStatus': statusString}})
    # Close Log File
    logFile.close()

    # Wait
    time.sleep(5)

498.0mV	Current: 9.16625	Voltage: 0.472655
497.0mV	Current: 9.16625	Voltage: 0.477037
496.0mV	Current: 9.16625	Voltage: 0.474846
495.0mV	Current: 9.16625	Voltage: 0.477037
494.0mV	Current: 9.16625	Voltage: 0.472655
493.0mV	Current: 9.16625	Voltage: 0.472655
492.0mV	Current: 9.16625	Voltage: 0.474846
491.0mV	Current: 9.16625	Voltage: 0.470464
490.0mV	Current: 9.16625	Voltage: 0.472655
489.0mV	Current: 9.16625	Voltage: 0.474846
488.0mV	Current: 9.16625	Voltage: 0.474846
487.0mV	Current: 9.16625	Voltage: 0.477037
486.0mV	Current: 9.16625	Voltage: 0.474846
485.0mV	Current: 9.16625	Voltage: 0.474846
484.0mV	Current: 9.16625	Voltage: 0.474846
483.0mV	Current: 9.16625	Voltage: 0.474846
482.0mV	Current: 9.16625	Voltage: 0.472655
481.0mV	Current: 9.16625	Voltage: 0.474846
480.0mV	Current: 9.16625	Voltage: 0.474846
479.0mV	Current: 9.16625	Voltage: 0.472655
478.0mV	Current: 9.16625	Voltage: 0.472655
477.0mV	Current: 9.16625	Voltage: 0.470464
476.0mV	Current: 9.16625	Voltage: 0.470464
475.0mV	Cur

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:20: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


KeyboardInterrupt: 

In [21]:
np.array(err).shape[0] > 1

False

In [ ]:
    # Check for impending jobs
#     nextJob = None
#     for job in jobcollection.find({}).limit(1).sort('startTime', ASCENDING):
#         nextJob = job
#     if (nextJob):
#         print("Next Job at " + nextJob['timeString'])
    
#     if (nextJob and (nextJob['startTime'] - datetime.utcnow()).seconds < 5):
#         # Mark previous current job as completed
#         if (currentJob and currentJob['inProgress']):
#             jobcollection.find_one_and_update({'_id': currentJob['_id']}, {'$set': {'inProgress': False, 'completed': True}})
        
#         # Next Job becomes current job
#         currentJob = nextJob
#         print("Starting Scheduled " + currentJob['type'])
#         jobcollection.find_one_and_update({'_id': currentJob['_id']}, {'$set': {'inProgress': True}})
# #         info = infocollection.find_one_and_update({},{'$set': {'command': currentJob['type']}})